In [4]:
pip install tensorflow numpy matplotlib scikit-learn seaborn opencv-python

You should consider upgrading via the 'c:\Users\Arya\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import ResNet50, VGG16, EfficientNetB0
from tensorflow.keras.layers import LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [16]:
# Define dataset path
train_dir = "train"
valid_dir = "valid"

# Define image size
img_size = (224, 224)
batch_size = 32

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2
)

valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Get class labels
class_labels = list(train_generator.class_indices.keys())
print(f"Class Labels: {class_labels}")

Found 3665 images belonging to 5 classes.
Found 916 images belonging to 5 classes.
Class Labels: ['Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']


In [17]:
def build_cnn():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(class_labels), activation='softmax')
    ])
    
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [18]:
def build_resnet():
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze pre-trained layers

    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(class_labels), activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [19]:
def build_vgg16():
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze layers

    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(class_labels), activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [20]:

def build_efficientnet():
    # Load EfficientNetB0 as the base model, excluding the top classification layers
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model layers

    # Build the custom model on top of EfficientNetB0
    model = Sequential([
        base_model,
        Flatten(),  # Flatten the output of the base model
        Dense(128, activation='relu'),  # Fully connected layer
        Dropout(0.5),  # Dropout to prevent overfitting
        Dense(len(class_labels), activation='softmax')  # Final output layer for classification
    ])

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
models = {
    "CNN": build_cnn(),
    "ResNet50": build_resnet(),
    "VGG16": build_vgg16(),
    "EfficientNetB0": build_efficientnet()
}

history_dict = {}
evaluation_results = {}

for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    
    history = model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs= 10
    )
    
    # Save model
    model.save(f"{model_name}_diabetic_retinopathy.h5")
    
    # Store training history
    history_dict[model_name] = history

    # Evaluate model
    loss, accuracy = model.evaluate(valid_generator)
    evaluation_results[model_name] = {"Loss": loss, "Accuracy": accuracy}

print("Training and Evaluation Complete!")


Training CNN...
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.1988 - loss: 1.6602 - val_accuracy: 0.2325 - val_loss: 1.5991
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.2179 - loss: 1.6068 - val_accuracy: 0.2314 - val_loss: 1.5981
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.2186 - loss: 1.6086 - val_accuracy: 0.2402 - val_loss: 1.5941
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.2298 - loss: 1.6009 - val_accuracy: 0.2336 - val_loss: 1.5899
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 115s 1s/step - accuracy: 0.2261 - loss: 1.6000 - val_accuracy: 0.2402 - val_loss: 1.5908
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.2272 - loss: 1.5976 - val_accuracy: 0.2293 - val_loss: 1.5861
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 115s 1000ms/step - accuracy: 0.2295 - loss: 1.5953 - val_accuracy: 0.2391 - val_loss: 1.5922
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.2345 - lo

29/29 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.2557 - loss: 1.5798

Training ResNet50...
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 335s 3s/step - accuracy: 0.1983 - loss: 1.9321 - val_accuracy: 0.2162 - val_loss: 1.6092
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 353s 3s/step - accuracy: 0.2207 - loss: 1.6096 - val_accuracy: 0.2162 - val_loss: 1.6086
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 336s 3s/step - accuracy: 0.2210 - loss: 1.6122 - val_accuracy: 0.2162 - val_loss: 1.6080
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 384s 3s/step - accuracy: 0.2171 - loss: 1.6093 - val_accuracy: 0.2162 - val_loss: 1.6075
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 341s 3s/step - accuracy: 0.2130 - loss: 1.6085 - val_accuracy: 0.2162 - val_loss: 1.6070
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 334s 3s/step - accuracy: 0.2312 - loss: 1.6094 - val_accuracy: 0.2162 - val_loss: 1.6065
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 316s 3s/step - accuracy: 0.2309 - loss: 1.6076 - val_accuracy: 0.2162 - val_loss: 1.6060


29/29 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.2177 - loss: 1.6035

Training VGG16...
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 704s 6s/step - accuracy: 0.2249 - loss: 1.7187 - val_accuracy: 0.2205 - val_loss: 1.6014
Epoch 2/10
 28/115 ━━━━━━━━━━━━━━━━━━━━ 7:07 5s/step - accuracy: 0.2453 - loss: 1.5957

In [ ]:
plt.figure(figsize=(12, 6))

for model_name, history in history_dict.items():
    plt.plot(history.history['accuracy'], label=f"{model_name} Train")
    plt.plot(history.history['val_accuracy'], label=f"{model_name} Validation")

plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training vs Validation Accuracy")
plt.legend()
plt.show()

In [13]:
def evaluate_model(model, generator, model_name):
    y_true = generator.classes
    y_pred = np.argmax(model.predict(generator), axis=1)
    report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
    
    # Convert to DataFrame and plot
    report_df = pd.DataFrame(report).transpose()
    plt.figure(figsize=(10, 5))
    sns.heatmap(report_df.iloc[:-1, :-1], annot=True, cmap="Blues", fmt=".2f")
    plt.title(f"Classification Report - {model_name}")
    plt.show()
    
    return report

# Evaluate and compare models
for model_name, model in models.items():
    print(f"\nClassification Report for {model_name}:")
    report = evaluate_model(model, valid_generator, model_name)


Classification Report for CNN:


c:\Users\Arya\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


29/29 ━━━━━━━━━━━━━━━━━━━━ 14s 484ms/step


c:\Users\Arya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Arya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Arya\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

NameError: name 'pd' is not defined

In [ ]:
def test_model(model_path, image_path):
    model = tf.keras.models.load_model(model_path)

    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)

    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    predicted_label = class_labels[predicted_class]

    # Show Image
    plt.imshow(cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title(f"Predicted: {predicted_label}")
    plt.show()

    return predicted_label

# Test Image
image_path = "dataset/test/sample_image.jpg"
test_model("ResNet50_diabetic_retinopathy.h5", image_path)